# Bert 연습해보기

## 임베딩을 사용하는 방법
1. 임베딩층(Embedding layer)을 랜덤 초기화하여 처음부터 학습하는 방법
2. 방대한 데이터로 Word2Vec등과 같은 임베딩 알고리즘으로 사전에 학습된 임베딩 벡터를 가져와 사용하는 방법

## 임베딩의 한계 및 해결책
* 하나의 단어가 하나의 벡터값으로 맵핑되어 문맥을 고려하지 못하는 한계가 있음<br>
 -> 사전 훈련된 언어모델인 ELMo나 BERT 등으로 해결

## Bert 전 모델들

### Semi-superviesed Sequence Learning<br>
:LSTM 언어 모델을 학습하고나서 이렇게 학습한 LSTM을 텍스트 분류에 추가 학습하는 방법

* 언어 모델은 주어진 텍스트로부터 이전 단어들로부터 다음 단어를 예측하도록 학습하므로 기본적으로 별도의 레이블이 부착되지 않은 텍스트 데이터로도 학습 가능(사전 훈련된 언어 모델의 강점은 학습 전 사람이 별도 레이블을 지정해줄 필요가 없다는 점)
* 레이블이 없는 데이터로 학습된 LSTM과 가중치가 랜덤으로 초기화 된 LSTM 두 가지를 두고, 텍스트 분류와 같은 문제를 학습하여 사전 훈련된 언어 모델을 사용한 전자의 경우가 더 좋은 성능을 얻을 수 있다는 가능성

![](https://wikidocs.net/images/page/108730/image1.PNG)

### ELMo 

* ELMo는 순방향 언어 모델과 역방향 언어 모델을 각각 따로 학습시킨 후에, 
이렇게 사전 학습된 언어 모델로부터 임베딩 값을 얻는다는 아이디어

* 이러한 임베딩은 문맥에 따라서 임베딩 벡터값이 달라지므로, 기존 워드 임베딩인 Word2Vec이나 GloVe 등이 
다의어를 구분할 수 없었던 문제점을 해결

![](https://wikidocs.net/images/page/108730/image2.PNG)

### GPT
-> LSTM 대신 Transformer사용

![](https://wikidocs.net/images/page/108730/image3.PNG)

![](https://wikidocs.net/images/page/108730/image4.PNG)

In [ ]:
우측에 있는 양방향 언어 모델은 지금까지 본 적 없던 형태의 언어 모델입니다. 
실제로 이렇게 구현하는 경우는 거의 없는데 그 이유가 무엇일까요? 
가령, 양방향 LSTM을 이용해서 우측과 같은 언어 모델을 만들었다고 해봅시다. 
초록색 LSTM 셀은 순방향 언어 모델로 <sos>를 입력받아 I를 예측하고, 그 후에 am을 예측합니다. 
그런데 am을 예측할 때, 출력층은 주황색 LSTM 셀인 역방향 언어 모델의 정보도 함께 받고있습니다. 
그런데 am을 예측하는 시점에서 역방향 언어 모델이 이미 관측한 단어는 a, am, I 이렇게 3개의 단어입니다. 
이미 예측해야하는 단어를 역방향 언어 모델을 통해 미리 관측한 셈이므로 언어 모델은 일반적으로 양방향으로 구현하지 않습니다.

하지만 문맥은 양방향이고 양방향을 구현하기 위해 마스크드 모델 탄생

## Bert(Bidirectional Encoder Representations from Transformers) 란?
 :BERT는 이전 챕터에서 배웠던 트랜스포머를 이용하여 구현되었으며,<br>
 위키피디아(25억 단어)와 BooksCorpus(8억 단어)와 같은 레이블이 없는 텍스트 데이터로 사전 훈련된 언어 모델

### Bert의 기본구조
: 트랜스포머의 인코더를 쌓아올린 구조입니다. Base 버전에서는 총 12개를 쌓았으며, Large 버전에서는 총 24개를 쌓음

![](https://wikidocs.net/images/page/35594/bartbase%EC%99%80large.PNG)

In [1]:
# !pip install sentence_transformers

     ---------------------------------------- 79.7/79.7 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 4.0/4.0 MB 31.7 MB/s eta 0:00:00
     ------------------------------------- 157.9/157.9 MB 19.9 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 62.9 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 34.9 MB/s eta 0:00:00
     ---------------------------------------- 77.9/77.9 KB ? eta 0:00:00
     ------------------------------------- 895.2/895.2 KB 55.3 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 42.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120748 sha256=38236afcebecd037357c30f959d0b002f24488725684c6337f64acb9b21754d3
  Stored in directory: c:\users\bitcamp\appdata\local\pip\cache\wheels\2b\11\3b\32a18fb9

In [4]:
import pandas as pd
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [8]:
doc = """
         (Reuters) - Activision Blizzard (NASDAQ:ATVI) is cooperating with federal investigations into trading by friends of its chief executive shortly before the gaming company disclosed its sale to Microsoft Corp (NASDAQ:MSFT), it said in a securities filing on Friday.

It received requests for information from the U.S. Securities and Exchange Commission and received a subpoena from a Department of Justice grand jury, the maker of "Call of Duty" said in an amended proxy filing.

The requests "appear to relate to their respective investigations into trading by third parties – including persons known to Activision Blizzard's CEO – in securities prior to the announcement of the proposed transaction," it said.

Microsoft in January agreed to acquire Activision for $95 a share, or $68.7 billion in total, in the biggest video-gaming industry deal in history.

The company did not name the parties, nor say whether the grand jury subpoena was directed at any employee.

The filing did not disclose when it received the subpoena or the SEC request for information.

Media moguls Barry Diller and David Geffen, and investor Alexander von Furstenberg, acquired share options after von Furstenberg met with Activision CEO Bobby Kotick and days before it disclosed the sale to Microsoft, the Wall Street Journal reported last month.

"Activision Blizzard has informed these authorities that it intends to be fully cooperative with these investigations," the company said.

Diller told Reuters last month that none of the three had any knowledge about a potential acquisition and had acted on the belief that Activision was undervalued and had the potential for going private or being acquired.

The amended proxy filing that included the information on its cooperation with the SEC and DOJ came after shareholders sued the company alleging omissions to a preliminary proxy on the sale.
    """

In [11]:
# 3개의 단어 묶음인 단어구 추출

n_gram_range = (3, 5)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc]) #ngram_rangetuple (min_n, max_n), default=(1, 1)
candidates = count.get_feature_names_out()

print('trigram 개수 :', len(candidates))
print('trigram 다섯개만 출력 :', candidates[:10])

trigram 개수 : 487
trigram 다섯개만 출력 : ['68 billion total' '68 billion total biggest'
 '68 billion total biggest video' '95 share 68' '95 share 68 billion'
 '95 share 68 billion total' 'acquire activision 95'
 'acquire activision 95 share' 'acquire activision 95 share 68'
 'acquired amended proxy']


In [13]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [14]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['known activision blizzard ceo securities', 'von furstenberg met activision ceo', 'activision blizzard ceo securities', 'activision blizzard ceo securities prior', 'blizzard ceo securities prior announcement']


## 다양한 키워드를 얻으려면?
* Max Sum Similarity: 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것
* Maximal Marginal Relevance :텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력

## Max Sum Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    
    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)
    
    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates

## Maximal Marginal Relevance

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
# 낮은 diversity
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

In [ ]:
# 높은 diversity
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)